In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristics OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [23]:
#decide which df to use
df = pd.read_csv("macro_2.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]
X_var_names = [ "Inflation Premium(%)", "90-Day AA Financial Commercial Paper Interest Rate",
               "CBOE Crude Oil ETF Volatility Index", "Consumer Sentiment", "Unemployment rate"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
0,AVY.N,2013-01-01,Materials,11.178,NaN,0.359537,0.17,0.02,22.54,76.67,7.733333
1,AVY.N,2013-04-01,Materials,2.482,NaN,0.360572,0.14,0.03,23.47,81.67,7.533333
2,AVY.N,2013-07-01,Materials,1.068,NaN,0.463789,0.13,0.04,24.10,81.57,7.233333
3,AVY.N,2013-10-01,Materials,8.095,NaN,0.441854,0.13,0.04,19.74,76.93,6.933333
4,AVY.N,2014-01-01,Materials,1.471,NaN,0.465460,0.13,0.05,19.56,80.93,6.666667
...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,1.527139,0.382645,0.14,0.06,43.04,69.90,4.200000
20116,POOL.OQ,2022-01-01,Retailing,40.267,1.515266,0.380946,0.47,0.32,52.70,63.13,3.800000
20117,POOL.OQ,2022-04-01,Retailing,34.342,2.325202,0.495588,1.39,0.61,50.46,57.87,3.600000
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.092855,0.423045,2.89,0.64,49.86,56.10,3.566667


#### OLS Regression

In [24]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()



In [25]:
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
12,AVY.N,2016-01-01,Materials,8.817,2.304994,0.420570,0.55,0.19,58.78,91.57,4.900000
13,AVY.N,2016-04-01,Materials,9.159,1.829833,0.383655,0.56,0.18,42.38,92.40,4.933333
14,AVY.N,2016-07-01,Materials,7.522,1.491713,0.406042,0.70,0.30,39.96,90.33,4.900000
15,AVY.N,2016-10-01,Materials,1.290,1.606498,0.454642,0.77,0.32,38.25,93.07,4.766667
16,AVY.N,2017-01-01,Materials,6.180,2.431507,0.444016,0.92,0.22,29.34,97.23,4.566667
...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,1.527139,0.382645,0.14,0.06,43.04,69.90,4.200000
20116,POOL.OQ,2022-01-01,Retailing,40.267,1.515266,0.380946,0.47,0.32,52.70,63.13,3.800000
20117,POOL.OQ,2022-04-01,Retailing,34.342,2.325202,0.495588,1.39,0.61,50.46,57.87,3.600000
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.092855,0.423045,2.89,0.64,49.86,56.10,3.566667


In [26]:
rescale = df_accuracy_new
rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["WACC Inflation Adjusted Risk Free Rate, (%)"] = MinMaxScaler().fit_transform(np.array(rescale["WACC Inflation Adjusted Risk Free Rate, (%)"]).reshape(-1,1))
rescale["Inflation Premium(%)"] = MinMaxScaler().fit_transform(np.array(rescale["Inflation Premium(%)"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = MinMaxScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["CPFFM"] = MinMaxScaler().fit_transform(np.array(rescale["CPFFM"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = MinMaxScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["Consumer Sentiment"] = MinMaxScaler().fit_transform(np.array(rescale["Consumer Sentiment"]).reshape(-1,1))
rescale["Unemployment rate"] = MinMaxScaler().fit_transform(np.array(rescale["Unemployment rate"]).reshape(-1,1))

rescale

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
12,AVY.N,2016-01-01,Materials,0.514221,0.646376,0.708327,0.108959,0.383562,0.422695,0.828158,0.141844
13,AVY.N,2016-04-01,Materials,0.516152,0.530450,0.614380,0.111380,0.369863,0.216379,0.847537,0.145390
14,AVY.N,2016-07-01,Materials,0.506909,0.447959,0.671354,0.145278,0.534247,0.185935,0.799206,0.141844
15,AVY.N,2016-10-01,Materials,0.471719,0.475963,0.795039,0.162228,0.561644,0.164423,0.863180,0.127660
16,AVY.N,2017-01-01,Materials,0.499331,0.677241,0.767996,0.198547,0.424658,0.052334,0.960308,0.106383
...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,0.561523,0.456602,0.611807,0.009685,0.205479,0.224682,0.322204,0.067376
20116,POOL.OQ,2022-01-01,Retailing,0.691809,0.453705,0.607483,0.089588,0.561644,0.346207,0.164137,0.024823
20117,POOL.OQ,2022-04-01,Retailing,0.658353,0.651306,0.899246,0.312349,0.958904,0.318027,0.041326,0.003546
20118,POOL.OQ,2022-07-01,Retailing,0.472921,0.838591,0.714626,0.675545,1.000000,0.310479,0.000000,0.000000


In [27]:
y = rescale["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]

X = rescale[X_var_names]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.027
Model:                                              OLS   Adj. R-squared:                  0.027
Method:                                   Least Squares   F-statistic:                     71.70
Date:                                  Tue, 28 Feb 2023   Prob (F-statistic):           2.88e-74
Time:                                          16:37:42   Log-Likelihood:                 11728.
No. Observations:                                 12860   AIC:                        -2.344e+04
Df Residuals:                                     12854   BIC:                        -2.340e+04
Df Model:                                             5                                         
Covariance Type:                              nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
const                                                  0.5625      0.005    105.626      0.000       0.552       0.573
Inflation Premium(%)                                  -0.0194      0.005     -4.152      0.000      -0.029      -0.010
90-Day AA Financial Commercial Paper Interest Rate    -0.0363      0.004     -8.713      0.000      -0.044      -0.028
CBOE Crude Oil ETF Volatility Index                   -0.0791      0.007    -12.060      0.000      -0.092      -0.066
Consumer Sentiment                                    -0.0310      0.003     -9.746      0.000      -0.037      -0.025
Unemployment rate                                      0.0428      0.007      6.288      0.000       0.029       0.056
==============================================================================
Omnibus:                     2117.684   Durbin-Watson:                   1.641
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19815.186
Skew:                           0.510   Prob(JB):                         0.00
Kurtosis:                       8.995   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### OLS Pooled Regression

In [28]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                                    object
Date                                                  datetime64[ns]
GICS Industry Group Name                                      object
Earnings Per Share - Actual Surprise                         float64
WACC Inflation Adjusted Risk Free Rate, (%)                  float64
Inflation Premium(%)                                         float64
90-Day AA Financial Commercial Paper Interest Rate           float64
CPFFM                                                        float64
CBOE Crude Oil ETF Volatility Index                          float64
Consumer Sentiment                                           float64
Unemployment rate                                            float64
dtype: object

In [29]:
#df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
12,AVY.N,2016-01-01,Materials,0.514221,0.646376,0.708327,0.108959,0.383562,0.422695,0.828158,0.141844
13,AVY.N,2016-04-01,Materials,0.516152,0.530450,0.614380,0.111380,0.369863,0.216379,0.847537,0.145390
14,AVY.N,2016-07-01,Materials,0.506909,0.447959,0.671354,0.145278,0.534247,0.185935,0.799206,0.141844
15,AVY.N,2016-10-01,Materials,0.471719,0.475963,0.795039,0.162228,0.561644,0.164423,0.863180,0.127660
16,AVY.N,2017-01-01,Materials,0.499331,0.677241,0.767996,0.198547,0.424658,0.052334,0.960308,0.106383
...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,0.561523,0.456602,0.611807,0.009685,0.205479,0.224682,0.322204,0.067376
20116,POOL.OQ,2022-01-01,Retailing,0.691809,0.453705,0.607483,0.089588,0.561644,0.346207,0.164137,0.024823
20117,POOL.OQ,2022-04-01,Retailing,0.658353,0.651306,0.899246,0.312349,0.958904,0.318027,0.041326,0.003546
20118,POOL.OQ,2022-07-01,Retailing,0.472921,0.838591,0.714626,0.675545,1.000000,0.310479,0.000000,0.000000


# with grouping


In [30]:
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Inflation Premium(%),90-Day AA Financial Commercial Paper Interest Rate,CPFFM,CBOE Crude Oil ETF Volatility Index,Consumer Sentiment,Unemployment rate
12,AVY.N,2016-01-01,Materials,0.514221,0.646376,0.708327,0.108959,0.383562,0.422695,0.828158,0.141844
13,AVY.N,2016-04-01,Materials,0.516152,0.530450,0.614380,0.111380,0.369863,0.216379,0.847537,0.145390
14,AVY.N,2016-07-01,Materials,0.506909,0.447959,0.671354,0.145278,0.534247,0.185935,0.799206,0.141844
15,AVY.N,2016-10-01,Materials,0.471719,0.475963,0.795039,0.162228,0.561644,0.164423,0.863180,0.127660
16,AVY.N,2017-01-01,Materials,0.499331,0.677241,0.767996,0.198547,0.424658,0.052334,0.960308,0.106383
...,...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,0.561523,0.456602,0.611807,0.009685,0.205479,0.224682,0.322204,0.067376
20116,POOL.OQ,2022-01-01,Retailing,0.691809,0.453705,0.607483,0.089588,0.561644,0.346207,0.164137,0.024823
20117,POOL.OQ,2022-04-01,Retailing,0.658353,0.651306,0.899246,0.312349,0.958904,0.318027,0.041326,0.003546
20118,POOL.OQ,2022-07-01,Retailing,0.472921,0.838591,0.714626,0.675545,1.000000,0.310479,0.000000,0.000000


In [31]:
from statsmodels.iolib.summary2 import summary_col
# Split the DataFrame into groups based on the stocks
df = df_accuracy_new[df_accuracy_new['GICS Industry Group Name'] == 'Technology Hardware & Equipment']
groups = df.groupby('Instrument')

# Define a function to perform OLS regression on each group
def ols_regression(group):
    # Define the dependent and independent variables
    if len(group) < 2:
        return None
    y = group['Earnings Per Share - Actual Surprise']
    X = group[X_var_names]

    # Add a constant to the independent variables
    X = sm.add_constant(X)

    # Fit the OLS model and return the results
    model = sm.OLS(y, X).fit()
    return model

# Apply the function to each group of data
results = groups.apply(ols_regression)
results = results.dropna()
# Collect the results into a new DataFrame

# Print the coefficients, t-values, and p-values
# print('Coefficients:')
# print(coefficients)
# print('\nT-values:')
# print(t_values)
# print('\nP-values:')
models = results.tolist()
summary = summary_col(models, stars=True, float_format='%0.2f', model_names=results.index.tolist())

# Print the combined summary table
summary

/opt/homebrew/Caskroom/mambaforge/base/envs/AnalystForecast/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
/opt/homebrew/Caskroom/mambaforge/base/envs/AnalystForecast/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/opt/homebrew/Caskroom/mambaforge/base/envs/AnalystForecast/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)


,AAPL.OQ,ANET.N,APH.N,CDW.OQ,CSCO.OQ,FFIV.OQ,GLW.N,HPE.N,HPQ.N,JNPR.N,KEYS.N,MSI.N,NTAP.OQ,STX.OQ,TDY.N,TEL.N,TRMB.OQ,WDC.OQ,ZBRA.OQ
const,0.61***,0.42***,0.56***,0.58***,0.47*,0.49***,0.43***,0.67***,0.55***,0.50***,0.68***,0.49***,0.61***,0.68,0.60***,0.57***,0.54***,0.47***,0.53***
,(0.06),(0.06),(0.04),(0.04),(0.06),(0.03),(0.03),(0.08),(0.08),(0.05),(0.07),(0.05),(0.09),(inf),(0.08),(0.04),(0.05),(0.11),(0.06)
Inflation Premium(%),-0.05,0.10*,0.00,-0.04,-0.02,0.01,0.05*,-0.09,-0.05,-0.01,-0.12*,0.02,0.01,-0.01,0.01,-0.04,0.03,0.19*,0.01
,(0.05),(0.05),(0.04),(0.03),(0.05),(0.03),(0.03),(0.07),(0.07),(0.04),(0.06),(0.04),(0.08),(inf),(0.07),(0.04),(0.05),(0.09),(0.05)
90-Day AA Financial Commercial Paper Interest Rate,-0.06,0.04,-0.02,-0.03,0.01,0.01,-0.02,-0.00,-0.02,0.03,0.00,-0.02,0.02,-0.27,-0.09,-0.02,-0.08*,-0.33***,-0.03
,(0.05),(0.05),(0.03),(0.03),(0.02),(0.02),(0.02),(0.08),(0.08),(0.04),(0.06),(0.04),(0.07),(inf),(0.06),(0.03),(0.04),(0.08),(0.05)
CBOE Crude Oil ETF Volatility Index,-0.13*,0.03,-0.12**,-0.09*,0.05,0.03,0.03,-0.35***,-0.04,-0.07,-0.21**,-0.01,-0.33***,-0.37,-0.15,-0.12**,-0.10,-0.06,-0.12
,(0.07),(0.07),(0.05),(0.04),(0.13),(0.04),(0.04),(0.10),(0.09),(0.06),(0.09),(0.06),(0.11),(inf),(0.10),(0.05),(0.07),(0.13),(0.07)
Consumer Sentiment,-0.06,0.06,-0.05*,-0.03,0.06,-0.02,0.03*,-0.05,-0.03,0.02,-0.04,0.03,-0.10*,-0.28,-0.00,-0.04,-0.02,0.02,-0.01
,(0.04),(0.04),(0.02),(0.02),(0.09),(0.02),(0.02),(0.05),(0.05),(0.03),(0.04),(0.03),(0.05),(inf),(0.05),(0.03),(0.03),(0.06),(0.04)
